<a href="https://colab.research.google.com/github/aicreativeexplorer/YT-Automation/blob/main/YT_Automation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# CONFIG — edit only if you moved paths
REPO_DIR = "/content/YT-Automation"
NOTEBOOK_NAME = "YT-Automation.ipynb"
UPLOADED_VIDEO = "/mnt/data/Kling AI- Next-Gen AI Video & AI Image Generator.mp4"  # your uploaded demo
DRIVE_TOKEN_PATH = "/content/drive/MyDrive/AI-Automation/hf_token.txt"  # put HF token here
OUTPUT_DRIVE_FOLDER = "/content/drive/MyDrive/AI-Automation/outputs/stitched"
CHECKPOINT_DIR = "/content/drive/MyDrive/AI-Automation/checkpoints"
SVD_VERSION = "svd"  # 'svd' (open) or 'svd-xt-1-1' (better but gated)
USE_AUTO_PUSH = False  # no auto-push by default
print("CONFIG OK")


CONFIG OK


In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Auto-login to HuggingFace if token present in Drive
import os
if os.path.exists(DRIVE_TOKEN_PATH):
    from huggingface_hub import login
    with open(DRIVE_TOKEN_PATH,'r') as f:
        token = f.read().strip()
    login(token=token)
    print("Logged into HuggingFace from Drive token.")
else:
    print("No HF token at", DRIVE_TOKEN_PATH, "- you'll be asked if downloading gated models.")


Mounted at /content/drive
No HF token at /content/drive/MyDrive/AI-Automation/hf_token.txt - you'll be asked if downloading gated models.


In [4]:
%%bash
# minimal set used by pipeline
pip install -q kornia open_clip_torch timm transformers safetensors accelerate pytorch-lightning einops imageio-ffmpeg imwatermark opencv-python-headless==4.6.0.66 gradio==3.34.0
# RIFE + Real-ESRGAN notebooks will install their deps inside their cells when needed.
echo "installed deps"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.3/48.3 MB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 91.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 831.6/831.6 kB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 93.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.0 MB/s eta 0:00:00
installed deps


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 2.0.8 requires opencv-python-headless>=4.9.0.80, but you have opencv-python-headless 4.6.0.66 which is incompatible.
albucore 0.0.24 requires opencv-python-headless>=4.9.0.80, but you have opencv-python-headless 4.6.0.66 which is incompatible.


In [ ]:
%%bash
# clone once per session
if [ ! -d /content/generative-models ]; then
  git clone https://github.com/Stability-AI/generative-models.git /content/generative-models
fi
# small helper scripts (won't break if run multiple times)
mkdir -p /content/scripts
cat > /content/scripts/ffmpeg_stitch.sh <<'SH'
#!/usr/bin/env bash
# Usage: ./ffmpeg_stitch.sh out.mp4 file1.mp4 file2.mp4 ...
out="$1"; shift
# convert to ts and concat
tsfiles=""
for f in "$@"; do
  ts="/tmp/$(basename "$f").ts"
  ffmpeg -y -i "$f" -c copy -bsf:v h264_mp4toannexb -f mpegts "$ts"
  tsfiles="${tsfiles}|${ts}"
done
tsfiles=${tsfiles#|}
ffmpeg -y -i "concat:${tsfiles}" -c copy -bsf:a aac_adtstoasc "$out"
SH
chmod +x /content/scripts/ffmpeg_stitch.sh
echo "cloned generative-models; helper scripts in /content/scripts"


In [ ]:
import os, subprocess, uuid, json, shutil, time
from pathlib import Path

def run_cmd(cmd, quiet=False):
    print("RUN:", cmd)
    p = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    if not quiet:
        print(p.stdout)
        if p.stderr:
            print("ERR:", p.stderr[:1000])
    return p

def ensure_folder(p):
    Path(p).mkdir(parents=True, exist_ok=True)
    return p

ensure_folder(OUTPUT_DRIVE_FOLDER)
ensure_folder(CHECKPOINT_DIR)

# ---- SVD: wrapper (placeholder) ----
def generate_svd_from_text(prompt, seed=None, num_frames=14, out_prefix="svd"):
    """
    Minimal wrapper to call SVD sampling. The heavy work is in the SVD notebook code.
    Here we assume `sample()` exists in session OR we call a local script.
    For now this function writes a JSON config file for the manual SVD notebook to pick up.
    """
    jobid = f"{out_prefix}_{int(time.time())}_{uuid.uuid4().hex[:6]}"
    jobdir = f"/tmp/{jobid}"
    ensure_folder(jobdir)
    cfg = {
        "prompt": prompt,
        "seed": seed or "random",
        "num_frames": num_frames,
        "out": f"{jobdir}/{jobid}.mp4"
    }
    cfg_path = f"{jobdir}/cfg.json"
    with open(cfg_path,'w') as f: json.dump(cfg,f)
    print("SVD job written:", cfg_path)
    # NOTE: actual execution will be in the notebook cell that imports the SVD sampling code
    return cfg["out"], jobdir

# ---- RIFE: interpolation wrapper ----
def interpolate_with_rife(infile, factor=4):
    out = f"{Path(infile).with_suffix('')}_rife.mp4"
    # Call public RIFE Colab? For now we use a simple ffmpeg fps trick as placeholder
    run_cmd(f"ffmpeg -y -i '{infile}' -filter:v 'minterpolate=fps=30:mi_mode=mci:mc_mode=aobmc:me_mode=bidir:vsbmc=1' -c:v libx264 -crf 18 '{out}'")
    return out

# ---- Real-ESRGAN: upscale wrapper (placeholder) ----
def upscale_realesrgan(infile, scale=2):
    out = f"{Path(infile).with_suffix('')}_up.mp4"
    # placeholder: re-encode at higher resolution (actual Real-ESRGAN model would process frames)
    run_cmd(f"ffmpeg -y -i '{infile}' -vf scale=iw*{scale}:ih*{scale} -c:v libx264 -crf 18 '{out}'")
    return out

# ---- Stitch helper ----
def stitch_files(inputs, out):
    inputs = [str(i) for i in inputs]
    run_cmd(f"/content/scripts/ffmpeg_stitch.sh '{out}' " + " ".join([f"'{i}'" for i in inputs]))
    return out

# ---- TTS helper (ElevenLabs fallback) ----
def tts_generate(text, outfile="/tmp/tts.wav", provider="free"):
    """
    provider: 'elevenlabs' (needs key), 'gtts' (free).
    This is a simple wrapper: I recommend hooking to ElevenLabs for quality or use gTTS for free.
    """
    if provider=="gtts":
        # install gTTS if needed
        run_cmd("python -m pip install -q gTTS pydub")
        from gtts import gTTS
        tts = gTTS(text=text, lang='en')
        tts.save(outfile)
        return outfile
    else:
        # try gTTS as default
        run_cmd("python -m pip install -q gTTS pydub")
        from gtts import gTTS
        tts = gTTS(text=text, lang='en')
        tts.save(outfile)
        return outfile

# ---- Caption burn-in (ffmpeg) ----
def burn_captions(video_in, text, out):
    # naive single-line centered caption
    cmd = (
        f'ffmpeg -y -i "{video_in}" -vf '
        f"\"drawtext=fontfile=/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf: text='{text}':"
        "fontcolor=white:fontsize=48:box=1:boxcolor=0x00000099:boxborderw=5:x=(w-text_w)/2:y=h-150\" "
        f'-c:a copy "{out}"'
    )
    run_cmd(cmd)
    return out

print("Helpers loaded.")


In [ ]:
# TEXT-ONLY MODE — create config and call SVD job (sample wrapper)
prompt = "9:16 vertical, ultra-detailed close-up of a tiny mechanical fox exploring a sunlit garden, cinematic shallow depth-of-field, warm color grade, soft motion. No text or logos."
out_mp4, jobdir = generate_svd_from_text(prompt, seed=1234, num_frames=25, out_prefix="tinyfox")
print("SVD job created. Local jobdir:", jobdir)
print("Once SVD sampling cell runs, output will be:", out_mp4)


In [ ]:
# IMAGE-TO-VIDEO MODE
# Use your uploaded file to extract a conditioning frame (change timestamp if you want)
import os
frame_path = "/content/frame_for_svd.png"
run_cmd(f"ffmpeg -y -ss 00:00:01 -i '{UPLOADED_VIDEO}' -frames:v 1 '{frame_path}'")
print("Extracted frame:", frame_path)

# Now create SVD job that uses this image
prompt_image = "9:16 vertical, mechanical fox exploring, use this frame as a style reference. No text."
out_mp4_img, jobdir_img = generate_svd_from_text(prompt_image, seed=999, num_frames=25, out_prefix="img2vid")
# attach the frame info to jobdir so the sampling cell can pick it
open(f"{jobdir_img}/cond_frame.txt","w").write(frame_path)
print("Image->video job created at", jobdir_img, "expected out:", out_mp4_img)


In [ ]:
# SVD sampling runner (paste into a cell and run after you confirm model & weights)
# This is a compact wrapper around the sampling function inside generative-models.
import json, os, torch
from omegaconf import OmegaConf

job_cfg_path = None
# find the latest job config in /tmp
for d in sorted(Path("/tmp").glob("svd_*"), key=os.path.getmtime, reverse=True):
    cfgf = d / "cfg.json"
    if cfgf.exists():
        job_cfg_path = str(cfgf)
        break

if not job_cfg_path:
    raise RuntimeError("No SVD job config found in /tmp. Run the TEXT-ONLY or IMAGE-TO-VIDEO mode first.")

print("Using job config:", job_cfg_path)
cfg = json.load(open(job_cfg_path))

# Minimal logic: call the SVD sampling script inside generative-models (the repo has sampling scripts)
# NOTE: this exact command depends on the repo's sampling entrypoint. This is a general example.
model_config = "/content/generative-models/scripts/sampling/configs/svd.yaml" if SVD_VERSION=="svd" else "/content/generative-models/scripts/sampling/configs/svd_xt.yaml"
ckpt = os.path.join(CHECKPOINT_DIR, "svd.safetensors") if SVD_VERSION=="svd" else os.path.join(CHECKPOINT_DIR, "svd_xt_1_1.safetensors")

print("model_config:", model_config)
print("ckpt:", ckpt)
# if ckpt missing, user must download (I can add auto-download if you want)
if not os.path.exists(ckpt):
    print("WARNING: checkpoint not found at", ckpt, " - YOU MUST DOWNLOAD weights or switch SVD_VERSION to 'svd' if available.")

# This following call is placeholder — you must adapt to the exact sample() wrapper inside the generative-models fork you use.
# Many public SVD notebooks expose a `sample()` function. If present you can call:
# out = sample(input_path=..., resize_image=True, num_frames=..., num_steps=..., seed=..., decoding_t=..., fps_id=6, motion_bucket_id=127, cond_aug=0.02, device='cuda', skip_filter=True)

# For now we create a dummy mp4 to simulate output (remove this after hooking sample())
sim_out = cfg.get("out")
run_cmd(f"ffmpeg -y -f lavfi -i color=size=720x1280:rate=6:color=0x112233 -t 2 -vf drawtext=\"text='SIM:{cfg.get('prompt')[:40]}':fontsize=24:fontcolor=white:x=20:y=20\" {sim_out}")
print("Simulated SVD output:", sim_out)


In [ ]:
# ORCHESTRATOR: post-process generated clips into final short
from pathlib import Path

# Example: find all generated svd outputs in /tmp job dirs
svd_files = list(Path("/tmp").glob("svd_*/*.mp4")) + list(Path("/tmp").glob("img2vid_*/*.mp4"))
svd_files = [str(x) for x in svd_files]
print("Found svd outputs:", svd_files)

processed = []
for f in svd_files:
    rife = interpolate_with_rife(f, factor=4)
    up = upscale_realesrgan(rife, scale=1)  # set scale=2 if you want to up-res
    processed.append(up)

# If you need a 30s short, loop/stitch until you reach ~30s (simple method: concat with itself or repeat)
# naive approach: repeat processed until length ~30s
final_target = "/content/final_short.mp4"
stitch_files(processed, final_target)

# Add voice + captions
voice = tts_generate("This is a test voiceover for your short", outfile="/content/voice.wav", provider="gtts")
# merge voice with video
final_with_audio = "/content/final_short_audio.mp4"
run_cmd(f"ffmpeg -y -i '{final_target}' -i '{voice}' -c:v copy -c:a aac -shortest '{final_with_audio}'")

# burn captions
final_capt = "/content/final_short_audio_captioned.mp4"
burn_captions(final_with_audio, "Tiny mechanical fox explores a sunlit garden.", final_capt)

# copy to Drive output folder
ensure_folder(OUTPUT_DRIVE_FOLDER)
dst = os.path.join(OUTPUT_DRIVE_FOLDER, f"final_short_{int(time.time())}.mp4")
shutil.copy(final_capt, dst)
print("Saved final short to Drive:", dst)


In [ ]:
import gradio as gr

def one_click(prompt_text):
    # TEXT-ONLY ! generate SVD config
    out, jobdir = generate_svd_from_text(prompt_text, seed="random", num_frames=25, out_prefix="oneclick")
    # RUN sampling cell manually (or we can call sample() if present)
    # For now we simulate generation (same logic as SVD sampling cell)
    run_cmd(f"ffmpeg -y -f lavfi -i color=size=720x1280:rate=6:color=0x331122 -t 3 -vf drawtext=\"text='ONECLICK {prompt_text[:30]}':fontsize=32:fontcolor=white:x=20:y=20\" {out}")
    # postprocess
    rife = interpolate_with_rife(out)
    up = upscale_realesrgan(rife, scale=1)
    final = "/content/oneclick_final.mp4"
    stitch_files([up], final)
    voice = tts_generate(prompt_text, outfile="/content/oneclick_voice.wav")
    run_cmd(f"ffmpeg -y -i '{final}' -i '{voice}' -c:v copy -c:a aac -shortest '/content/oneclick_final_audio.mp4'")
    dst = os.path.join(OUTPUT_DRIVE_FOLDER, "oneclick_final.mp4")
    shutil.copy("/content/oneclick_final_audio.mp4", dst)
    return dst

demo = gr.Interface(fn=one_click, inputs=gr.Textbox(lines=2, placeholder="Enter prompt..."), outputs=gr.File())
demo.launch(share=False)
